In [1]:
###############################################################################
### pthon imports                                                           ###
###############################################################################

from os import path
from requests import get
from bs4 import BeautifulSoup
import os
import pandas as pd
import re

In [2]:
def find_file(
    file_name='path/file.txt',
    cache=False,
    cache_age=None
    ):

    if cache==False:
        return None
    
    return path.exists(file_name)

In [14]:
def get_soup(
    url='https://github.com/search?o=desc&p=1&q=advent+of+code&s=stars&type=Repositories',
    headers={'User-Agent': 'Nothing suspicious'},
    file_name='soupfiles/gitsearch.txt',
    cache=False,
    cache_age=None,
    soup_slurper='*'
):
    # if we already have the data, read it locally
    file_found = find_file(file_name=file_name, cache=cache, cache_age=cache_age)
    if file_found:
        with open(file_name) as f:
            return BeautifulSoup(f.read())

    # otherwise go fetch the data
    response = get(url, headers=headers)
    soup = BeautifulSoup(response.text)
    slurps = soup.select(soup_slurper)

    # save it for next time
    with open(file_name, 'w') as f:
        f.write(str(slurps[0]))
        if len(slurps)>1:
            for slurp in slurps[1:]:
                f.write('\n' + str(slurp))
        
    with open(file_name) as f:    
        soup = BeautifulSoup(f.read())
#     pd.to_csv(slurps, header=None, index=False)

    return soup

In [43]:
soup = get_soup(url='https://github.com/search?o=desc&p=2&q=advent+of+code&s=stars&type=Repositories', soup_slurper='.repo-list-item a')

In [44]:
print(soup)

<html><body><a class="v-align-middle" data-hydro-click='{"event_type":"search_result.click","payload":{"page_number":2,"per_page":10,"query":"advent of code","result_position":1,"click_id":76759168,"result":{"id":76759168,"global_relay_id":"MDEwOlJlcG9zaXRvcnk3Njc1OTE2OA==","model_name":"Repository","url":"https://github.com/mfikes/advent-of-code"},"client_id":null,"originating_request_id":"EE85:591E:349F2F9:5DA4BC7:5DEFCCD0","originating_url":"https://github.com/search?o=desc&amp;p=2&amp;q=advent+of+code&amp;s=stars&amp;type=Repositories","referrer":null,"user_id":null}}' data-hydro-click-hmac="43ae6e5f3f0c75e2afd0e9f99a1119790d44e6fa17204c6fdf86afc3ec180f93" href="/mfikes/advent-of-code">mfikes/<em>advent</em>-<em>of</em>-<em>code</em></a>
<a class="muted-link" href="/mfikes/advent-of-code/stargazers">
<svg aria-label="star" class="octicon octicon-star" height="16" role="img" version="1.1" viewbox="0 0 14 16" width="14"><path d="M14 6l-4.9-.64L7 1 4.9 5.36 0 6l3.6 3.26L2.67 14 7 11.6

In [45]:
re_url = re.compile(r'"url"\:"(.+?)"')
urllist = re_url.findall(str(soup))

urllist

['https://github.com/mfikes/advent-of-code',
 'https://github.com/dlew/aoc-2017',
 'https://github.com/mstksg/advent-of-code-2017',
 'https://github.com/kodsnack/advent_of_code_2017',
 'https://github.com/Diggsey/aoc2018',
 'https://github.com/mhanberg/advent-of-code-elixir-starter',
 'https://github.com/mstksg/advent-of-code-2019',
 'https://github.com/borkdude/advent-of-cljc',
 'https://github.com/BartoszMilewski/AofC2017',
 'https://github.com/zero-to-mastery/coding_challenge-24']